```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Interpolation errors

Goal of this notebook: To $N(0,I)$Â random numbers on a coarse grid, apply the SOAR function, and then interpolate the result down to the fine computational grid.


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import OceanStateNoise, GPUDrifterCollection


In [ ]:
%cuda_context_handler gpu_ctx
gpu_stream = cuda.Stream()

In [ ]:
def imshow(im, interpolation="None", title=None, figsize=(4,4)):
    fig = plt.figure(figsize=figsize)
    plt.imshow(im, interpolation=interpolation, origin='lower')
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
def imshow3(eta, hu, hv, interpolation="None", title=None, interior=False):
    fig, axs = plt.subplots(1,3, figsize=(12,4))
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower')
    axs[0].set_title("eta")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower')
    axs[1].set_title("hu")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower')
    axs[2].set_title("hv")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()

def imshow6(eta1, hu1, hv1, eta2, hu2, hv2, interpolation="None", title=None):
    fig, axs = plt.subplots(3,3, figsize=(12,13))
    
    range_eta = max(np.max(np.abs(eta1)), np.max(np.abs(eta2)))
    range_huv = max(np.max(np.abs(hu1)), np.max(np.abs(hu2)), np.max(np.abs(hv1)), np.max(np.abs(hv2)))
    
    eta_im = axs[0,0].imshow(eta1, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[0,0].set_title("eta 1")
    plt.colorbar(eta_im, ax=axs[0,0])
    
    hu_im = axs[0,1].imshow(hu1, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[0,1].set_title("hu 1")
    plt.colorbar(hu_im, ax=axs[0,1])

    hv_im = axs[0,2].imshow(hv1, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[0,2].set_title("hv 1")
    plt.colorbar(hv_im, ax=axs[0,2])

    
    eta_im2 = axs[1,0].imshow(eta2, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[1,0].set_title("eta 2")
    plt.colorbar(eta_im, ax=axs[1,0])
    
    hu_im2 = axs[1,1].imshow(hu2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1,1].set_title("hu 2")
    plt.colorbar(hu_im, ax=axs[1,1])

    hv_im2 = axs[1,2].imshow(hv2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1,2].set_title("hv 2")
    plt.colorbar(hv_im, ax=axs[1,2])
    
    
    eta_im3 = axs[2,0].imshow(eta1 - eta2, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[2,0].set_title("diff eta")
    plt.colorbar(eta_im, ax=axs[2,0])
    
    hu_im3 = axs[2,1].imshow(hu1 - hu2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2,1].set_title("diff hu")
    plt.colorbar(hu_im, ax=axs[2,1])

    hv_im3 = axs[2,2].imshow(hv1 - hv2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2,2].set_title("diff hv")
    plt.colorbar(hv_im, ax=axs[2,2])

    if title is not None:
        plt.suptitle(title, fontsize=18)
    plt.tight_layout()   

# Testing the new class


In [ ]:
if 'newnoise' in globals():
    newnoise.cleanUp()
reload(OceanStateNoise)
nx, ny = 210, 210
dx, dy = 100.0, 100.0
interpolation_factor = 7

newnoise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, \
                                           nx, ny, dx, dy, \
                                           Common.BoundaryConditions(2,2,2,2),
                                           False,
                                           interpolation_factor=interpolation_factor)

dataShape = (ny+4, nx+4)
buf = np.zeros(dataShape, dtype=np.float32)
H_buffer = np.ones((ny+5, nx+5), dtype=np.float32) * 10

sim = CDKLM16.CDKLM16(gpu_ctx, buf.copy(),buf.copy(),buf.copy(), H_buffer, \
                      nx, ny, dx, dy, 0.5, 9.81, 0.002, 0  )

newnoise.perturbSim(sim)

coarse_vals = newnoise.getCoarseBuffer()
#imshow(coarse_vals, title="Coarse GPU")

#imshow(coarse_vals[2:-2, 2:-2], title="Coarse GPU (interior only)")

eta, hu, hv = sim.download(interior_domain_only=False)
imshow3(eta, hu, hv, title="Ocean state GPU", interior=True)

e = np.zeros(dataShape, dtype=np.float32)
u = np.zeros(dataShape, dtype=np.float32)
v = np.zeros(dataShape, dtype=np.float32)
#v = np.zeros((28, 28), dtype=np.float32)
newnoise.perturbOceanStateCPU(e,u,v,H_buffer, sim.f, \
                              use_existing_GPU_random_numbers=True,
                              ghost_cells_x=2, ghost_cells_y=2)
#newnoise.perturbEtaCPU(e, True, ghost_cells_x=2, ghost_cells_y=2)
imshow3(e,u,v, title="Ocean state CPU", interior=True)

imshow3(coarse_vals[2:-2, 2:-2], eta[2:-2, 2:-2], e[2:-2, 2:-2], title="eta {coarse, GPU, CPU}")

imshow(coarse_vals[2:-2, 2:-2], title="coarse_eta[2:-2, 2:-2]", figsize=(12,12))
imshow(e[2:-2, 2:-2], title="fine_eta[2:-2, 2:-2]", figsize=(12,12))
imshow(u[2:-2, 2:-2], title="fine_hu[2:-2, 2:-2]", figsize=(12,12))

fig = plt.figure(figsize=(10,4))
plt.plot(eta[2:-2, 20]/np.max(eta))
plt.plot( hu[2:-2, 20]/np.max( hu))
plt.grid()

In [ ]:
print(coarse_vals.shape)
print(eta.shape)
#imshow(coarse_vals[2:-2,2:-2] - eta[2:-2,2:-2])
imshow(e[2:-2,2:-2] - eta[2:-2,2:-2])
imshow3(e - eta, u - hu, v - hv)



## Case study

Investigate how perturbations with the various interpolation factors are preserved in a deterministic setting

In [ ]:
sim_args = {
    "gpu_ctx": gpu_ctx,
    "nx": 210, "ny": 210,
    "dx": 1000.0, "dy": 1000.0,
    "dt": 0.5,
    "g": 9.81,
    "f": 1.4e-4,
    "coriolis_beta": 0.0,
    "r": 0.0,
    "rk_order": 2,
    "small_scale_perturbation_amplitude": 0.01,
    "write_netcdf": False, 
    "small_scale_perturbation": True,
    "small_scale_perturbation_interpolation_factor": 1

}

iterations = 100
stepsize = 100*sim_args["dt"]

ghosts = [2,2,2,2] # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
    
sim_args["H"]    = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 23
sim_args["eta0"] = np.zeros(dataShape, dtype=np.float32)
sim_args["hu0"]  = np.zeros(dataShape, dtype=np.float32)
sim_args["hv0"]  = np.zeros(dataShape, dtype=np.float32)
sim_args["boundary_conditions"] = Common.BoundaryConditions(2,2,2,2)

print (sim_args)


In [ ]:
def run_sim(sim, iterations, stepsize):
    fig = plt.figure()
    plotter = PlotHelper.PlotHelper.fromsim(sim, fig)
    
    def animate(i):
        t = 0.0
        if i > 0:
            t = sim.step(stepsize, apply_stochastic_term=False)
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        plotter.plot(eta1, u1, v1)
        if i%20 == 0:
            print("animation: " + str(100*i/iterations) + "%")
        
        fig.suptitle("Interpolation factor " + str(sim.small_scale_perturbation_interpolation_factor) + \
                     ", Time = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    anim = animation.FuncAnimation(fig, animate, range(iterations), interval=100)
    plt.close(fig)
    return anim

In [ ]:
interpolation_case_study=[{"interpolation_factor": 1},
                          {"interpolation_factor": 3},
                          {"interpolation_factor": 5},
                          {"interpolation_factor": 7} ]

for case in interpolation_case_study:
    sim_args["small_scale_perturbation_interpolation_factor"] = case["interpolation_factor"]
    case["sim"] = CDKLM16.CDKLM16(**sim_args)
    case["sim"].step(sim_args["dt"])
    case["init_eta"], case["init_hu"], case["init_hv"] = case["sim"].download(interior_domain_only=True)

In [ ]:
run_sim(interpolation_case_study[0]["sim"], iterations, stepsize)

In [ ]:
run_sim(interpolation_case_study[1]["sim"], iterations, stepsize)

In [ ]:
run_sim(interpolation_case_study[2]["sim"], iterations, stepsize)

In [ ]:
run_sim(interpolation_case_study[3]["sim"], iterations, stepsize)

In [ ]:
for case in interpolation_case_study:
    case["final_eta"], case["final_hu"], case["final_hv"] = case["sim"].download(interior_domain_only=True)
    imshow6(case["init_eta"], case["init_hu"], case["init_hv"],\
            case["final_eta"], case["final_hu"], case["final_hv"] ,\
            title="Interpolation factor " + \
                   str(case["sim"].small_scale_perturbation_interpolation_factor))
    

In [ ]:
CRASH

## Investigating $\Delta \eta$ mean value

The trend in the perturbation is that the $\eta$-field is perturbed with negative mean. This will lead to a significant loss of mass across several timesteps.

In [ ]:
def evalMean(sim):
    eta, hu, hv = sim.download()
    eta = eta[2:-2, 2:-2]
    meanEta = np.mean(eta)
    return meanEta

if 'sim1' in globals():
    sim1.cleanUp()
if 'noise1' in globals():
    noise1.cleanUp()
if 'sim2' in globals():
    sim2.cleanUp()
if 'noise2' in globals():
    noise2.cleanUp()
reload(CDKLM16)
reload(OceanStateNoise)
# Creating sim
sim1 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt/3, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False,
                      small_scale_perturbation_interpolation_factor=5)
sim2 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt/3, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False,
                      small_scale_perturbation_interpolation_factor=5)


q0 = 100*dx/100000
noise1 = OceanStateNoise.OceanStateNoise.fromsim(sim1, soar_q0=q0)
noise2 = OceanStateNoise.OceanStateNoise.fromsim(sim2, soar_q0=q0)
meanT = 5000
means1 = np.zeros(meanT)
means2 = np.zeros(meanT)
for i in range(meanT):
    for j in range(50):
        noise1.perturbSim(sim1)
        noise2.perturbSim(sim2)
        noise2.resetSeed()
    means1[i] = evalMean(sim1)
    means2[i] = evalMean(sim2)
    if i%500 == 0:
        print("iteration i = " + str(i))
fig = plt.figure(figsize=(6,3))
plt.plot(means1, label="no reset")
plt.plot(means2, label="reset")
plt.legend(loc=0)
plt.grid()
print (sim1.nx, sim1.ny)

In [ ]:
# Looking at autocorrelation within a cell:

def autocorr(x, k, mean=None, var=None):
    R = np.zeros(k)
    n = len(x)
    if mean is None:
        mean = np.mean(x)
    if var is None:
        var = np.var(x)
    for i in range(k):
        r = 0.0
        for t in range(n-i):
            r += (x[t] - mean)*(x[t+i] - mean)
        R[i] = r/((n-i)*var)
    return R

if 'sim2' in globals():
    sim2.cleanUp()
if 'noise2' in globals():
    noise2.cleanUp()
reload(CDKLM16)
reload(OceanStateNoise)
reload(Common)
# Creating sim
sim2 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False,
                      small_scale_perturbation_interpolation_factor=5)



q0 = 100*dx/100000
noise2 = OceanStateNoise.OceanStateNoise.fromsim(sim2, soar_q0=q0)
noise2.resetSeed()
autocorrN = 10000
for rounds in range(5):
    for i in range(100000):
        noise2.generateNormalDistribution()
    print ("round " + str(rounds))
    u = np.zeros((autocorrN, 9))
    for i in range(autocorrN):
        noise2.generateNormalDistribution()
        generatedU = noise2.random_numbers.download(noise2.gpu_stream)
        for j in range(9):
            u[i,j] = generatedU[j,j]


    fig = plt.figure(figsize=(4,2))
    for j in range(9):
        lag = autocorr(u[:,j], 100)
        plt.plot(np.abs(lag))
        print ("mean row " + str(j), np.mean(u[:,j]))
    print ("total mean: ", np.mean(u))
    print ("min/max: ", np.min(u), np.max(u))
    plt.title("autocorr")
    plt.grid()
    
    fig = plt.figure(figsize=(4,2))
    cov = np.cov(u, rowvar=False)
    print( cov.shape)
    plt.imshow(cov)
#np.u


In [ ]:
print 1103515245
print 2**31*0.51386433
print 2**31
print np.float64(2147483648.0)

print np.int16(1103515245)
print (0x7fffffff)

a = np.double(3.14)
print a
print isinstance(a, np.float64)
type(a)

In [ ]:
print(nx, ny)
print(123%100)